In [3]:
from PIL import Image 
image = Image.open('messi.png')

In [4]:
import argparse
import os
import pathlib
import subprocess
import sys
from typing import Callable, Union

import dlib
import huggingface_hub
import numpy as np
import PIL.Image
import torch
import torch.nn as nn
import torchvision.transforms as T


In [5]:
def denormalize(tensor: torch.Tensor) -> torch.Tensor:
        return torch.clamp((tensor + 1) / 2 * 255, 0, 255).to(torch.uint8)
def postprocess(tensor: torch.Tensor) -> np.ndarray:
        tensor = denormalize(tensor)
        return tensor.cpu().numpy().transpose(1, 2, 0)

In [6]:
from models.psp import pSp

In [7]:
def load_e4e() -> nn.Module:
        ckpt_path = huggingface_hub.hf_hub_download('public-data/e4e',
                                                    'e4e_ffhq_encode.pt')
        ckpt = torch.load(ckpt_path, map_location='cpu')
        opts = ckpt['opts']
        opts['device'] = "cpu"
        opts['checkpoint_path'] = ckpt_path
        if 'output_size' not in opts:
                opts['output_size'] = 1024
        print(ckpt_path)
        opts = argparse.Namespace(**opts)
        model = pSp(opts)
        model.to('cpu')
        model.eval()
        return model

In [8]:
e4e = load_e4e()

/home/ishant/.cache/huggingface/hub/models--public-data--e4e/snapshots/e1f997577fec5f953f98997ffcc65e4ae00ad2cd/e4e_ffhq_encode.pt


Exception: Encoder4Editing is not a valid encoders

In [19]:
def create_transform():
        transform = T.Compose([
            T.Resize(256),
            T.CenterCrop(256),
            T.ToTensor(),
            T.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5]),
        ])
        return transform
transform = create_transform()

In [23]:
def reconstruct_face(image: PIL.Image.Image):
        input_data = transform(image).unsqueeze(0).to("cpu")
        print(input_data.shape)
        reconstructed_images, latents = e4e(input_data,
                                                 randomize_noise=False,
                                                 return_latents=True)
        print(reconstructed_images.shape)
        print(latents[0].shape)
        reconstructed = torch.clamp(reconstructed_images[0].detach(), -1, 1)
        reconstructed = postprocess(reconstructed)
        return reconstructed, latents[0]

In [24]:
r , latent = reconstruct_face(image)

torch.Size([1, 3, 256, 256])
torch.Size([1, 18, 512])
torch.Size([1, 128, 64, 64])


ValueError: axes don't match array

: 

In [9]:
%wget http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2

UsageError: Line magic function `%wget` not found.
